In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

## creating the model

In [2]:
df = pd.read_csv('../week_3/data-week-3.csv')

df.columns = df.columns.str.lower().str.replace(' ', '_')

df = df.applymap(
    lambda x: x.lower().replace(' ', '_') if (isinstance(x, str)) else x
)

df['totalcharges']= pd.to_numeric(
    df.totalcharges, errors='coerce'
).fillna(0)

df.churn = (df.churn == 'yes').astype(int)

In [10]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=1)
y_test = df_test.churn.values

In [11]:
numerical = ['tenure', 'monthlycharges', 'totalcharges']
categorical = [
    'gender', 'seniorcitizen', 'partner', 'dependents', 'phoneservice', 
    'multiplelines', 'internetservice', 'onlinesecurity', 'onlinebackup',
    'deviceprotection', 'techsupport', 'streamingtv', 'streamingmovies', 
    'contract', 'paperlessbilling', 'paymentmethod'
]

In [12]:
def train(df_train, y_train, C=1.0):
    dicts = df_train[categorical + numerical].to_dict(orient='records')

    dv = DictVectorizer(sparse=False)
    X_train = dv.fit_transform(dicts)

    model = LogisticRegression(C=C, max_iter=1000)
    model.fit(X_train, y_train)
    
    return dv, model

In [13]:
def predict(df, dv, model):
    dicts = df[categorical + numerical].to_dict(orient='records')

    X = dv.transform(dicts)
    y_pred = model.predict_proba(X)[:, 1]

    return y_pred

In [14]:
C = 1.0
n_splits = 5

In [15]:
kfold = KFold(n_splits=n_splits, shuffle=True, random_state=1)

scores = []

for train_idx, val_idx in kfold.split(df_full_train):
    df_train = df_full_train.iloc[train_idx]
    df_val = df_full_train.iloc[val_idx]

    y_train = df_train.churn.values
    y_val = df_val.churn.values

    dv, model = train(df_train, y_train, C=C)
    y_pred = predict(df_val, dv, model)

    auc = roc_auc_score(y_val, y_pred)
    scores.append(auc)

print('C=%s %.3f +- %.3f' % (C, np.mean(scores), np.std(scores)))

C=1.0 0.840 +- 0.008


In [17]:
dv, model = train(df_full_train, df_full_train.churn.values, C=1.0)
y_pred = predict(df_test, dv, model)

auc = roc_auc_score(y_test, y_pred)
auc

0.8572386167896259

## savind the model

In [18]:
## saving using pickle
import pickle

In [19]:
output_file = f'model_C={C}.bin'
output_file

'model_C=1.0.bin'

In [21]:
f_out = open(output_file, 'wb') ## wb is for write binary
pickle.dump((dv, model), f_out)
f_out.close()

In [22]:
with open(output_file, 'wb') as f_out:
    pickle.dump((dv, model), f_out)  # better way to save it

## load the model

In [1]:
## restaring the kernel
import pickle

In [2]:
input_file = 'model_C=1.0.bin'

In [3]:
with open(input_file, 'rb') as f_in:  ## read bites
    dv, model = pickle.load(f_in)  # better way to save it

In [6]:
dv, model

(DictVectorizer(sparse=False), LogisticRegression(max_iter=1000))

In [7]:
customer = {
    'gender': 'female',
    'seniorcitizen': 0,
    'partner': 'yes',
    'dependents': 'no',
    'phoneservice': 'no',
    'multiplelines': 'no_phone_service',
    'internetservice': 'dsl',
    'onlinesecurity': 'no',
    'onlinebackup': 'yes',
    'deviceprotection': 'no',
    'techsupport': 'no',
    'streamingtv': 'no',
    'streamingmovies': 'no',
    'contract': 'month-to-month',
    'paperlessbilling': 'yes',
    'paymentmethod': 'electronic_check',
    'tenure': 1,
    'monthlycharges': 29.85,
    'totalcharges': 29.85
}

In [8]:
X = dv.transform([customer])

In [10]:
model.predict_proba(X)[0, 1]

0.6363584152729249